In [1]:
import pyomo.environ as pyo
import json
from datetime import timedelta
import polars as pl
from polars import selectors as cs
from polars  import col as c
import os 
import numpy as np
import math
import tqdm
from datetime import timedelta, datetime, timezone
# from optimization_model.optimizaztion_pipeline import first_stage_pipeline
from typing_extensions import Optional
from data_display.input_data_plots import plot_basin_height_volume_table

from data_display.baseline_plots import *
from utility.pyomo_preprocessing import extract_optimization_results, linear_interpolation_for_bound, arange_float, linear_interpolation_using_cols 
from smallflex_data_schema import SmallflexInputSchema
from utility.pyomo_preprocessing import *
from config import settings
from general_function import pl_to_dict, pl_to_dict_with_tuple, build_non_existing_dirs, generate_log, duckdb_to_dict

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
from plotly.graph_objects import Figure

from plotly.subplots import make_subplots

import networkx as nx
from baseline_model.baseline_input import BaseLineInput
from baseline_model.first_stage.first_stage_pipeline import BaselineFirstStage
from baseline_model.second_stage.second_stage_pipeline import BaselineSecondStage
COLORS = px.colors.qualitative.Plotly

log = generate_log(name="test")

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"


In [2]:
output_file_names: dict[str, str] = json.load(open(settings.FILE_NAMES)) # type: ignore
PARALLEL = False
YEARS = [2020, 2021, 2022, 2023]
TURBINE_FACTORS = {0.75, 0.85, 0.95}

SIMULATION_SETTING = [
    {"quantile": 0, "buffer": 0.2, "powered_volume_enabled": True},
    # {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True},
    # {"quantile": 0.25, "buffer": 0.3, "powered_volume_enabled": False},
    # {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True, "global_price": True},
    # {"quantile": 0.25, "buffer": 0.3, "powered_volume_enabled": False, "global_price": True},
]

YEARS = 2020
TURBINE_FACTORS = 0.75
# SIMULATION_SETTING = [{"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True}]

REAL_TIMESTEP = timedelta(hours=1)
FIRST_STAGE_TIMESTEP = timedelta(days=1)
SECOND_STAGE_TIME_SIM = timedelta(days=4)
TIME_LIMIT = 20 # in seconds
VOLUME_FACTOR = 1e-6




baseline_input: BaseLineInput = BaseLineInput(
    input_schema_file_name=output_file_names["duckdb_input"],
    real_timestep=REAL_TIMESTEP,
    year=YEARS,
    max_alpha_error=2,
    hydro_power_mask = c("name").is_in(["Aegina hydro"]),
    volume_factor=VOLUME_FACTOR
)
first_stage: BaselineFirstStage = BaselineFirstStage(
    input_instance=baseline_input,
    timestep=FIRST_STAGE_TIMESTEP,
    turbine_factor=TURBINE_FACTORS
)
first_stage.solve_model()


second_stage: BaselineSecondStage = BaselineSecondStage(
        input_instance=baseline_input, 
        first_stage=first_stage, 
        timestep=timedelta(days=4), 
        time_limit=TIME_LIMIT,
        model_nb=0,
        is_parallel=PARALLEL,
        **SIMULATION_SETTING[0]
    )

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 16/16 [00:02<00:00,  5.43it/s]
Solving first stage optimization problem: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.35s/it]


In [17]:
second_stage.power_performance_table

[{'H': 0,
  'B': 0,
  'power_performance': shape: (37, 8)
  ┌────────┬─────────────┬─────────────┬────────────┬────────────┬─────────┬────────────┬────────────┐
  │ height ┆ flow_pumped ┆ flow_turbin ┆ electrical ┆ electrical ┆ volume  ┆ alpha_turb ┆ alpha_pump │
  │ ---    ┆ ---         ┆ ed          ┆ _power_pum ┆ _power_tur ┆ ---     ┆ ined       ┆ ed         │
  │ f64    ┆ f64         ┆ ---         ┆ ped        ┆ bined      ┆ f64     ┆ ---        ┆ ---        │
  │        ┆             ┆ f64         ┆ ---        ┆ ---        ┆         ┆ f64        ┆ f64        │
  │        ┆             ┆             ┆ f64        ┆ f64        ┆         ┆            ┆            │
  ╞════════╪═════════════╪═════════════╪════════════╪════════════╪═════════╪════════════╪════════════╡
  │ 2350.0 ┆ -2.7354     ┆ 2.3248      ┆ -12.108    ┆ 7.2683     ┆ 1.39706 ┆ 3.126419   ┆ 4.426409   │
  │ 2351.0 ┆ -2.72304    ┆ 2.32998     ┆ -12.0828   ┆ 7.30702    ┆ 1.62089 ┆ 3.136087   ┆ 4.437247   │
  │ 2352.0 ┆ -2

In [8]:
second_stage.generate_index()

In [16]:
second_stage.index["state"].unnest("flow_pumped")

S,height,min,avg,max,diff,flow_turbined,electrical_power_pumped,electrical_power_turbined,volume,alpha_turbined,alpha_pumped,H,B,S_BH,BS,HS
u32,struct[4],f64,f64,f64,f64,struct[4],struct[4],struct[4],struct[4],struct[4],struct[4],i32,i64,list[i64],list[i64],list[i64]
0,"{2350.0,2356.5,2363.0,13.0}",2.7354,2.65536,2.57532,-0.16008,"{2.3248,2.35822,2.39164,0.06684}","{12.108,11.942,11.776,-0.332}","{7.2683,7.51979,7.77128,0.50298}","{1.39706,3.51551,5.63396,4.2369}","{3.126419,3.187886,3.249352,0.122932}","{4.426409,4.499522,4.572636,0.146226}",0,0,"[0, 0, … 0]","[0, 0]","[0, 0]"
1,"{2363.0,2369.0,2375.0,12.0}",2.57532,2.50126,2.4272,-0.14812,"{2.39164,2.42177,2.4519,0.06026}","{11.776,11.617,11.458,-0.318}","{7.77128,8.00284,8.2344,0.46312}","{5.63396,8.66228,11.6906,6.05664}","{3.249352,3.303864,3.358375,0.109023}","{4.572636,4.646651,4.720666,0.14803}",0,0,"[0, 0, … 1]","[0, 1]","[0, 1]"
2,"{2375.0,2380.5,2386.0,11.0}",2.4272,2.35142,2.27564,-0.15156,"{2.4519,2.47887,2.50584,0.05394}","{11.458,11.3053,11.1526,-0.3054}","{8.2344,8.44617,8.65794,0.42354}","{11.6906,14.9669,18.2432,6.5526}","{3.358375,3.40674,3.455105,0.09673}","{4.720666,4.810764,4.900863,0.180197}",0,0,"[0, 0, … 2]","[0, 2]","[0, 2]"
3,null,null,null,null,null,null,null,null,"{0.0,1.0,1.0,1.0}",null,null,null,1,"[null, 1, … 3]","[1, 3]","[null, 3]"
